# Week 1 : Systematically Improving Your RAG Application

## Benchmarking Retrieval Methods

> If you have not already, please run the `1. synthetic_questions.ipynb` notebook to generate the synthetic questions we'll be using in this notebook before proceeding.

After generating the synthetic questions, we'll use them to benchmark the recall and mrr of various retrievel methods. We want to do so because these metrics give us an objective measure of how well a retrieval method is performing.

This allows us to make an informed decision about whether a retrieval method might be worth the cost and latency to implement. For instance, if we get a 1% improvement in recall@10 but latency increases by 10x, we might not want to use that method.

We'll do this in two steps

1. First, we'll take our original dataset of sql snippets from `567-labs/bird-rag` and ingest it into a local lancedb instance.
2. Then we'll show you how to measure recall and mrr at different levels of k

Recall that in our original dataset, each row had the following columns

- `id`: The unique identifier for each SQL snippet
- `query`: The SQL snippet
- `difficulty`: The difficulty of the question

When we query our database with a synthetic question, we'll retrieve a list of sql snippets and their ids that match the query. Our goal here is to verify that we're able to retrieve the correct id for each question. 

We'll be using `braintrust` to collect the data and discuss the trade offs of each method. We like `braintrust` because it allows us to easily run multiple experiments using a simple `Eval` object and share the results easily if you're working with a team.


## Setting up Our Rag Pipeline

In this example, we're using a local `lancedb` instance. We're doing so because of 3 reasons.

1. LanceDB handles the embeddings of our data for us 
2. It provides embedding search, hybrid search and other re-ranking methods all within a single api.
3. We can use Pydantic to define our table schema and easily ingest our data.

This makes it quick and easy for us to compare the performance of each method.

In [1]:
# import datasets
# import lancedb
# from lancedb.pydantic import LanceModel, Vector
# from lancedb.embeddings import get_registry

# # Create LanceDB Instance
# db = lancedb.connect("./lancedb")

# # Define and create Table using Pydantic
# func = get_registry().get("openai").create(name="text-embedding-3-small")


# class Chunk(LanceModel):
#     id: str
#     query: str = func.SourceField()
#     vector: Vector(func.ndims()) = func.VectorField()


# table = db.create_table("chunks", schema=Chunk, mode="overwrite")

# # Ingest dataset into table
# dataset = datasets.load_dataset("567-labs/bird-rag")["train"]
# formatted_dataset = [{"id": item["id"], "query": item["query"]} for item in dataset]

# table.add(formatted_dataset)

# table.create_fts_index("query", replace=True)
# print(f"{table.count_rows()} chunks ingested into the database")

In [1]:
import lancedb

db = lancedb.connect("./lancedb")
table = db.open_table("chunks")

## Defining Metrics

Let's now start by evaluating the retrieval performance of our model. We'll do so by measuring the recall and MRR at different levels of k.

$$ \text{Recall} = \frac{\text{Number of Relevant Items Retrieved}}{\text{Total Number of Relevant Items}} $$ 

$$ \text{MRR} = \frac{\sum_{i=1}^{n} \frac{1}{rank(i)}}{n} $$ 

As models improve, their context window and reasoning abilities improve. This means that their ability to select relevant information in response to a user query will improve. By optimizing for recall, we ensure that the language model has access to all necessary information, which can lead to more accurate and reliable generated responses.

MRR@K is a useful metric if we want to display retrieved results as citations to users. We normally show a smaller list of retrieved results to users and we want to make sure that the correct result is ranked highly during retrieval so that it's more likely to be selected.

<TODO: Jason please verify or add on here >

In [2]:
def calculate_mrr(predictions: list[str], gt: list[str]):
    mrr = 0
    for label in gt:
        if label in predictions:
            # Find the relevant item that has the smallest index
            mrr = max(mrr, 1 / (predictions.index(label) + 1))
    return mrr


def calculate_recall(predictions: list[str], gt: list[str]):
    # Calculate the proportion of relevant items that were retrieved
    return len([label for label in gt if label in predictions]) / len(gt)

### Computing Metrics

We want to determine whether we can retrieve the relevant SQL snippet for each synthetic question. Remember that in our earlier notebook, we generated the question below for the corresponding SQL snippet.

Question : Which schools in France are locally funded and have a greater difference between their total K-12 
enrollment and enrollment for ages 5-17 compared to the average difference for locally funded schools?

Snippet 
```sql
SELECT T2.School, T2.DOC FROM frpm AS T1 INNER JOIN schools AS T2 ON T1.CDSCode = T2.CDSCode WHERE
T2.FundingType = 'Locally funded' AND (T1.`Enrollment (K-12)` - T1.`Enrollment (Ages 5-17)`) > (SELECT 
AVG(T3.`Enrollment (K-12)` - T3.`Enrollment (Ages 5-17)`) FROM frpm AS T3 INNER JOIN schools AS T4 ON T3.CDSCode = 
T4.CDSCode WHERE T4.FundingType = 'Locally funded')
```

Let's see this in action by fetching the top 25 results from our database that match this query using embedding search. Notice here that we're using the chunk id to calculate recall and mrr.


In [3]:
from rich import print

question = "Which schools in France are locally funded and have a greater difference between their total K-12 enrollment and enrollment for ages 5-17 compared to the average difference for locally funded schools?"

query = "SELECT T2.School, T2.DOC FROM frpm AS T1 INNER JOIN schools AS T2 ON T1.CDSCode = T2.CDSCode WHERE T2.FundingType = 'Locally funded' AND (T1.`Enrollment (K-12)` - T1.`Enrollment (Ages 5-17)`) > (SELECT AVG(T3.`Enrollment (K-12)` - T3.`Enrollment (Ages 5-17)`) FROM frpm AS T3 INNER JOIN schools AS T4 ON T3.CDSCode = T4.CDSCode WHERE T4.FundingType = 'Locally funded')"

retrieved_items = table.search(question).limit(25).to_list()

for item in retrieved_items[:2]:
    print(item["id"], item["query"])

28 SELECT T2.School, T2.DOC FROM frpm AS T1 INNER JOIN schools AS T2 ON T1.CDSCode = T2.CDSCode WHERE 
T2.FundingType = 'Locally funded' AND (T1.`Enrollment (K-12)` - T1.`Enrollment (Ages 5-17)`) > (SELECT 
AVG(T3.`Enrollment (K-12)` - T3.`Enrollment (Ages 5-17)`) FROM frpm AS T3 INNER JOIN schools AS T4 ON T3.CDSCode = 
T4.CDSCode WHERE T4.FundingType = 'Locally funded')

14 SELECT T1.NCESSchool FROM schools AS T1 INNER JOIN frpm AS T2 ON T1.CDSCode = T2.CDSCode ORDER BY T2.`Enrollment
(Ages 5-17)` DESC LIMIT 5

We can see that the first item is a perfect match for our desired chunk. We're going to calculate the recall@25 and MRR@25 for this specific snippet. To do so, we'll use its id. The id of this specific snippet is 28. You can verify it by looking at the filtered subset of the hugging face dataset [here](https://huggingface.co/datasets/567-labs/bird-rag/viewer/default/train?q=T4.FundingType)

In [5]:
predicted_ids = [item["id"] for item in retrieved_items]
ground_truth_ids = ["28"]  # The ID of the expected SQL snippet

# Calculate metrics
mrr_score = calculate_mrr(predicted_ids, ground_truth_ids)
recall_score = calculate_recall(predicted_ids, ground_truth_ids)
rank_position = predicted_ids.index("28") + 1  # Adding 1 because index starts at 0

print(f"MRR@25: {mrr_score}")
print(f"Recall@25: {recall_score}")
print(f"Relevant item found at rank: {rank_position}")

MRR@25: 1.0

Recall@25: 1.0

Relevant item found at rank: 1

This is a simple query since we have many of the specific column values in the snippet listed in the question - Eg. (K-12) and (Ages 5-17). But we're able to see how well our retrieval system is performing in this case with an objective measure.

We use a wrapper to compute metrics for multiple subsets of the retrieved items without having to hard code functions that take in a subset of the retrieved items. This makes it easy to vary the size of k and the metrics we want to use instead of hard coding them.

<TODO: Jason please verify this>

In [4]:
# Define the names for our metrics
eval_metrics = [["mrr", calculate_mrr], ["recall", calculate_recall]]

# Define the different sizes of k that we want to compute the metrics at
sizes = [1, 3, 5, 10, 15, 25]

# Define wrapper functions that will take a subset of k items and calculate the relevant metric
metrics = {}
for metric_name, metric_fn in eval_metrics:
    for size in sizes:
        key = f"{metric_name}@{size}"
        metrics[key] = lambda predictions, gt, m=metric_fn, s=size: m(
            predictions[:s], gt
        )

We can see this in action below with a simple example.

Our MRR@1 and Recall@1 will be 0 since there are no relevant items at k = 1. However, at k=3, we see that we have `a` in the top 3 retrieved items. This gives us a MRR@3 of 1/3 and a Recall@3 of 1/2 ( since we retrieved 1 of 2 relevant items).

Once we look at the MRR@5, we see that we have `a` and `b` in the top 5 retrieved items. This gives us a MRR@5 of 1/3 and a Recall@5 of 2/2 (since we retrieved all of the relevant items). We can see that we have the same values for MRR@10 and Recall@10.

In [7]:
labels = ["a", "b"]
preds = ["x", "y", "a", "c", "b", "f", "g", "h", "i", "k"]

{metric: score_fn(preds, labels) for metric, score_fn in metrics.items()}

{'mrr@1': 0,
 'mrr@3': 0.3333333333333333,
 'mrr@5': 0.3333333333333333,
 'mrr@10': 0.3333333333333333,
 'recall@1': 0.0,
 'recall@3': 0.5,
 'recall@5': 1.0,
 'recall@10': 1.0}

## Running your first Benchmark

We use braintrust here because it's easy to setup complex experiments. We can use the `Eval` object to define a single experiment and run it with different configurations. 

We can do so by defining a 

- `Task` : This is a function that takes in the input and returns an expected output
- `Scorer` : This is a function that looks at the output and compares it to the expected output and returns a score. In our case, since we're comparing the recall and mrr at different levels of k, we'll be returning a list of scores.

Since our task object is going to be running the retrieval with different methods, we'll also be redefining our `retrieve` function to take in additional parameters to be reconfigured for each experiment.

We implemented (1) in the previous section. Now, we want to use a single function to retrieve the top k items from our database with different retrieval methods because it's significantly easier to modify a single function with configuration parameters than to keep track of individual functions for each method.

In [5]:
from braintrust import Score
from lancedb.rerankers import CohereReranker
import lancedb
from typing import Literal

db = lancedb.connect("./lancedb")
table = db.open_table("chunks")
# Configure a cohere reranker
reranker = CohereReranker(model_name="rerank-multilingual-v3.0", column="query")


def retrieve(
    question: str,
    max_k=25,
    mode: Literal["vector", "fts", "hybrid"] = "vector",
    use_reranker: bool = False,
):
    results = table.search(question, query_type=mode).limit(max_k)
    if use_reranker:
        results = results.rerank(reranker=reranker)
    return [result for result in results.to_list()]


# Similar to our previous section, we can use the id of each item to compute the recall and MRR metrics.
def evaluate_braintrust(input, output, **kwargs):
    predictions = [item["id"] for item in output]
    labels = [kwargs["metadata"]["chunk_id"]]
    return [
        Score(
            name=metric,
            score=score_fn(predictions, labels),
            metadata={"query": input, "result": output, **kwargs["metadata"]},
        )
        for metric, score_fn in metrics.items()
    ]

> Note: We will remove a chunk of these once braintrust is functional again. For now, we're manually logging and calculating the results

We want to understand how each method performs when we vary the size of retrieve items with respect to recall and mrr. This helps us understand if a method is worth the cost and latency to implement. 

More practically, if a cheaper method has a recall@10 that's equivalent to a more expensive's method recall@5, then it might be worth it to start with the cheaper method first. Using these objective metrics, we can make an informed decision about which method to use. Let's see how we can do so with a simple example below.


In [12]:
import signal

# Define a class here to handle potential stalled retrieval requests - each request should take ~ 1-2 seconds at most so we can just kill the request after 5 seconds ( Noticed that some of my requests were stalling for lancedb)
class Timeout:
    def __init__(self, seconds=1, error_message='Timeout'):
        self.seconds = seconds
        self.error_message = error_message

    def handle_timeout(self, signum, frame):
        raise TimeoutError(self.error_message)

    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)

    def __exit__(self, type, value, traceback):
        signal.alarm(0)

In [13]:
from itertools import product
import braintrust
import time
from tqdm import tqdm
import pandas as pd

dataset = [
    item
    for item in braintrust.init_dataset(
        project="Text-2-SQL", name="Bird-Bench-Questions"
    )
]

results = {}
for query_mode, use_reranker in product(["vector", "fts", "hybrid"], [True, False]):
    experiment_results = []
    for data in tqdm(
        dataset,
        desc=f"Running with query mode {query_mode} and use reranker {use_reranker}",
    ):
        input_data = data["input"]

        start_time = time.time()
        
        for _ in range(3):
            try:
                with Timeout(seconds=5):
                    retrieved_results = retrieve(
                        input_data, 50, mode=query_mode, use_reranker=use_reranker
                    )
                    break
            except TimeoutError:
                continue
        end_time = time.time()


        predictions = [result["id"] for result in retrieved_results]
        labels = [data["metadata"]["chunk_id"]]
        scores = {
            metric: score_fn(predictions, labels)
            for metric, score_fn in metrics.items()
        }
        scores["avg_time"] = end_time - start_time
        experiment_results.append(scores)

    results[(use_reranker, query_mode)] = pd.DataFrame(experiment_results)
    # experiment.summarize(summarize_scores=False)

Running with query mode vector and use reranker True: 100%|██████████| 290/290 [04:12<00:00,  1.15it/s]
Running with query mode vector and use reranker False: 100%|██████████| 290/290 [02:34<00:00,  1.87it/s]
Running with query mode fts and use reranker True: 100%|██████████| 290/290 [01:42<00:00,  2.82it/s]
Running with query mode fts and use reranker False: 100%|██████████| 290/290 [00:04<00:00, 72.16it/s]
Running with query mode hybrid and use reranker True: 100%|██████████| 290/290 [04:10<00:00,  1.16it/s]
Running with query mode hybrid and use reranker False: 100%|██████████| 290/290 [02:39<00:00,  1.82it/s]


In [14]:
# Create a DataFrame with the mean values for all query parameters
mean_results = pd.DataFrame(
    {
        "query parameter": [str(key) for key in results.keys()],
        **{
            metric: [results[key][metric].mean() for key in results.keys()]
            for metric in results[list(results.keys())[0]].columns
        },
    }
)

# Set 'query parameter' as the index
mean_results.set_index("query parameter", inplace=True)

mean_results = mean_results.round(2)

# Display the results as a table
display(mean_results)

,mrr@1,mrr@3,mrr@5,mrr@10,mrr@15,mrr@25,recall@1,recall@3,recall@5,recall@10,recall@15,recall@25,avg_time
query parameter,,,,,,,,,,,,,
"(True, 'vector')",0.60,0.68,0.69,0.70,0.70,0.70,0.60,0.78,0.82,0.89,0.92,0.96,0.87
"(False, 'vector')",0.46,0.58,0.60,0.61,0.62,0.62,0.46,0.73,0.79,0.91,0.95,0.98,0.53
"(True, 'fts')",0.55,0.58,0.59,0.59,0.59,0.60,0.55,0.62,0.65,0.70,0.72,0.74,0.35
"(False, 'fts')",0.10,0.15,0.18,0.20,0.20,0.21,0.10,0.22,0.33,0.46,0.52,0.61,0.01
"(True, 'hybrid')",0.60,0.67,0.68,0.69,0.69,0.70,0.60,0.77,0.80,0.88,0.91,0.96,0.86
"(False, 'hybrid')",0.45,0.54,0.55,0.57,0.57,0.58,0.45,0.64,0.72,0.83,0.89,0.94,0.55


Now that we've computed the metrics above, we can see that

1. A Re-Ranker improves recall@k for all query types, especially when k is small. For recall, this improvement is only seen for FTS and Hybrid Search. For embedding search, using a re-ranker resulted in slightly worse performance. This improvement comes at the cost of increased latency of around 300ms, which is huge for FTS.
2. Embedding Search has around 3-13% higher recall than Hybrid Search for all values of k. The two have similar MRR@k for all values of k
3. A Re-Ranker only improves MRR but not recall for embedding search, indicating that the additional latency might not be worth it if MRR is not a priority

What's really surprising in this case is that embedding search alone outperforms hybrid search. Now that we've seen the individual performance, the next step is to do a sensitivity analysis. We do so because we want to make sure the differences we observe are robust and not due to chance. 

This is especially important if implementing these methods will require additional engineering effort. We'll explore some useful techniques such as bootstrapping and t-tests to do so in the next notebook `3. Visualising Results`.
